In [1]:
import pickle as pkl
import os
#
import pandas as pd
import numpy as np
import random
#
from dfmf import dfmf

Nimfa is library was not found. Certain initialization algorithms will not be available.


In [2]:
import sys
sys.path.append("..")

In [3]:
import pprint
import numpy as np
import pickle as pkl
import time
import itertools
import os

In [4]:
base_dname = "./../../../sample_data_NCMF/ncmf_sim_data/"
in_dir = base_dname + "cmf/"
out_dir_base = base_dname + "dfmf/out/"

In [5]:
for dataset_name in ['dt1', 'ds1', 'ds2', 'ds3', 'dn1', 'dn2', 'dn3']:
    print("dataset_name: ",dataset_name)
    print("#")
    #
    fname = in_dir + dataset_name + "/0.csv"
    X0 = pd.read_csv(fname,header=None).values
    #
    fname = in_dir + dataset_name + "/1.csv"
    X1 = pd.read_csv(fname,header=None).values
    #
    fname = in_dir + dataset_name + "/2.csv"
    X2 = pd.read_csv(fname,header=None).values
    #
    fname = in_dir + dataset_name + "/3.csv"
    X3 = pd.read_csv(fname,header=None).values
    #
    fname = in_dir + dataset_name + "/4.csv"
    X4 = pd.read_csv(fname,header=None).values
    #
    out_dir = out_dir_base + dataset_name + "/"
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    #
    print("X0.shape: ",X0.shape)
    print("X1.shape: ",X1.shape)
    print("X2.shape: ",X2.shape)
    print("X3.shape: ",X3.shape)
    print("X4.shape: ",X4.shape)
    #
    G = {"e0":["X0","X1","X2"],\
         "e1":["X0","X3"],\
         "e2":["X1","X4"],\
         "e3":["X2"],\
         "e4":["X3"],\
         "e5":["X4"]}
    #
    X_data = {
        "X0":X0,
        "X1":X1,
        "X2":X2,
        "X3":X3,
        "X4":X4}
    #
    X_meta = {"X0":["e0","e1"],\
         "X1":["e0","e2"],\
         "X2":["e0","e3"],\
         "X3":["e4","e1"],\
         "X4":["e5","e2"]}
    #
    X_val = {}
    #
    #DFMF
    dict_e_size = {}
    for e_id in G.keys():
        x_id = G[e_id][0]
        if X_meta[x_id][0] == e_id:
            dict_e_size[e_id] = X_data[x_id].shape[0]
        else:
            dict_e_size[e_id] = X_data[x_id].shape[1]
    #
    print("#")
    print("dict_e_size: ")
    print("#")
    for e_id in dict_e_size.keys():
        print("e_id: ",e_id,", size: ",dict_e_size[e_id])
    print("#") 
    #
    dict_e_code = {}
    dict_e_code_reverse = {}
    count_idx = 0
    for e_id in dict_e_size.keys():
        dict_e_code[e_id] = count_idx
        dict_e_code_reverse[count_idx] = e_id
        count_idx+=1
    #
    list_e_id = list(dict_e_size.keys())
    num_cluster = 100
    dict_num_clusters = {}
    for e_id in list_e_id:
        dict_num_clusters[e_id] = num_cluster
    #
    R = {}
    num_entities = len(list(dict_e_size.keys()))
    R['shape'] = (num_entities,num_entities)
    for x_id in X_meta.keys():
        R[(dict_e_code[X_meta[x_id][0]],dict_e_code[X_meta[x_id][1]])] = X_data[x_id]

    Theta = {}

    ns = []
    cs = []
    for i in np.arange(num_entities):
        ns.append(dict_e_size[dict_e_code_reverse[i]])
        cs.append(dict_num_clusters[dict_e_code_reverse[i]])
    #
    max_iter = 50
    dict_G_factors, dict_S_clust_asso = dfmf(R, Theta, ns, cs, max_iter=max_iter)
    #
    # get I and C matrices
    dict_I = {}
    dict_C = {}
    dict_pred_labels = {}
    for e_id in G.keys():
        cur_e_code = dict_e_code[e_id]
        cur_C = dict_G_factors[(cur_e_code,cur_e_code)]
        cur_I = np.zeros(cur_C.shape)
        cur_lables_list = []
        for i in np.arange(cur_C.shape[0]):
            j = np.argmax(cur_C[i,:])
            cur_I[i,j] = 1
            cur_lables_list.append(j)
        dict_I[e_id] = cur_I
        dict_C[e_id] = cur_C
        dict_pred_labels[e_id] = np.array(cur_lables_list)
    #
    #get A matrices
    dict_A = {}
    for x_id in X_meta.keys():
        row_e_id = X_meta[x_id][0]
        col_e_id = X_meta[x_id][1]
        row_e_code = dict_e_code[row_e_id] 
        col_e_code = dict_e_code[col_e_id] 
        cur_A = dict_S_clust_asso[(row_e_code,col_e_code)]
        dict_A[x_id] = cur_A

    #compute X reconstructions X_prime
    print("X reconstruction -> X_prime: ")
    X_prime = {}
    for x_id in X_meta.keys():
        print("x_id: ",x_id)
        cur_x_prime = np.dot(dict_I[X_meta[x_id][0]],np.dot(dict_A[x_id],dict_I[X_meta[x_id][1]].T))
        X_prime[x_id] = cur_x_prime
    #
    #
    out_dir = out_dir_base + dataset_name + "/"
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    #
    fname_out = out_dir + "dict_out_dfmf.pkl"
    print("fname_out: ",fname_out)
    pkl.dump(X_prime,open(fname_out,"wb"))

('dataset_name: ', 'dt1')
#


INFO:Medusa - Collective MF:[Random] Factor initialization
INFO:Medusa - Collective MF:[0] Iteration
INFO:Medusa - Collective MF:[0] Updating S
INFO:Medusa - Collective MF:[0] Updating G


('X0.shape: ', (200, 500))
('X1.shape: ', (200, 300))
('X2.shape: ', (200, 400))
('X3.shape: ', (700, 500))
('X4.shape: ', (600, 300))
#
dict_e_size: 
#
('e_id: ', 'e5', ', size: ', 600)
('e_id: ', 'e4', ', size: ', 700)
('e_id: ', 'e1', ', size: ', 500)
('e_id: ', 'e0', ', size: ', 200)
('e_id: ', 'e3', ', size: ', 400)
('e_id: ', 'e2', ', size: ', 300)
#


INFO:Medusa - Collective MF:[0] Solving for constraint matrices
INFO:Medusa - Collective MF:[1] Iteration
INFO:Medusa - Collective MF:[1] Updating S
INFO:Medusa - Collective MF:[1] Updating G
INFO:Medusa - Collective MF:[1] Solving for constraint matrices
INFO:Medusa - Collective MF:[2] Iteration
INFO:Medusa - Collective MF:[2] Updating S
INFO:Medusa - Collective MF:[2] Updating G
INFO:Medusa - Collective MF:[2] Solving for constraint matrices
INFO:Medusa - Collective MF:[3] Iteration
INFO:Medusa - Collective MF:[3] Updating S
INFO:Medusa - Collective MF:[3] Updating G
INFO:Medusa - Collective MF:[3] Solving for constraint matrices
INFO:Medusa - Collective MF:[4] Iteration
INFO:Medusa - Collective MF:[4] Updating S
INFO:Medusa - Collective MF:[4] Updating G
INFO:Medusa - Collective MF:[4] Solving for constraint matrices
INFO:Medusa - Collective MF:[5] Iteration
INFO:Medusa - Collective MF:[5] Updating S
INFO:Medusa - Collective MF:[5] Updating G
INFO:Medusa - Collective MF:[5] Solving 

INFO:Medusa - Collective MF:[42] Solving for constraint matrices
INFO:Medusa - Collective MF:[43] Iteration
INFO:Medusa - Collective MF:[43] Updating S
INFO:Medusa - Collective MF:[43] Updating G
INFO:Medusa - Collective MF:[43] Solving for constraint matrices
INFO:Medusa - Collective MF:[44] Iteration
INFO:Medusa - Collective MF:[44] Updating S
INFO:Medusa - Collective MF:[44] Updating G
INFO:Medusa - Collective MF:[44] Solving for constraint matrices
INFO:Medusa - Collective MF:[45] Iteration
INFO:Medusa - Collective MF:[45] Updating S
INFO:Medusa - Collective MF:[45] Updating G
INFO:Medusa - Collective MF:[45] Solving for constraint matrices
INFO:Medusa - Collective MF:[46] Iteration
INFO:Medusa - Collective MF:[46] Updating S
INFO:Medusa - Collective MF:[46] Updating G
INFO:Medusa - Collective MF:[46] Solving for constraint matrices
INFO:Medusa - Collective MF:[47] Iteration
INFO:Medusa - Collective MF:[47] Updating S
INFO:Medusa - Collective MF:[47] Updating G
INFO:Medusa - Collec

...

INFO:Medusa - Collective MF:[Random] Factor initialization
INFO:Medusa - Collective MF:[0] Iteration


('dataset_name: ', 'dn3')
#
('X0.shape: ', (200, 500))
('X1.shape: ', (200, 300))
('X2.shape: ', (200, 400))
('X3.shape: ', (700, 500))
('X4.shape: ', (600, 300))
#
dict_e_size: 
#
('e_id: ', 'e5', ', size: ', 600)
('e_id: ', 'e4', ', size: ', 700)
('e_id: ', 'e1', ', size: ', 500)
('e_id: ', 'e0', ', size: ', 200)
('e_id: ', 'e3', ', size: ', 400)
('e_id: ', 'e2', ', size: ', 300)
#


INFO:Medusa - Collective MF:[0] Updating S
INFO:Medusa - Collective MF:[0] Updating G
INFO:Medusa - Collective MF:[0] Solving for constraint matrices
INFO:Medusa - Collective MF:[1] Iteration
INFO:Medusa - Collective MF:[1] Updating S
INFO:Medusa - Collective MF:[1] Updating G
INFO:Medusa - Collective MF:[1] Solving for constraint matrices
INFO:Medusa - Collective MF:[2] Iteration
INFO:Medusa - Collective MF:[2] Updating S
INFO:Medusa - Collective MF:[2] Updating G
INFO:Medusa - Collective MF:[2] Solving for constraint matrices
INFO:Medusa - Collective MF:[3] Iteration
INFO:Medusa - Collective MF:[3] Updating S
INFO:Medusa - Collective MF:[3] Updating G
INFO:Medusa - Collective MF:[3] Solving for constraint matrices
INFO:Medusa - Collective MF:[4] Iteration
INFO:Medusa - Collective MF:[4] Updating S
INFO:Medusa - Collective MF:[4] Updating G
INFO:Medusa - Collective MF:[4] Solving for constraint matrices
INFO:Medusa - Collective MF:[5] Iteration
INFO:Medusa - Collective MF:[5] Updating

INFO:Medusa - Collective MF:[42] Updating S
INFO:Medusa - Collective MF:[42] Updating G
INFO:Medusa - Collective MF:[42] Solving for constraint matrices
INFO:Medusa - Collective MF:[43] Iteration
INFO:Medusa - Collective MF:[43] Updating S
INFO:Medusa - Collective MF:[43] Updating G
INFO:Medusa - Collective MF:[43] Solving for constraint matrices
INFO:Medusa - Collective MF:[44] Iteration
INFO:Medusa - Collective MF:[44] Updating S
INFO:Medusa - Collective MF:[44] Updating G
INFO:Medusa - Collective MF:[44] Solving for constraint matrices
INFO:Medusa - Collective MF:[45] Iteration
INFO:Medusa - Collective MF:[45] Updating S
INFO:Medusa - Collective MF:[45] Updating G
INFO:Medusa - Collective MF:[45] Solving for constraint matrices
INFO:Medusa - Collective MF:[46] Iteration
INFO:Medusa - Collective MF:[46] Updating S
INFO:Medusa - Collective MF:[46] Updating G
INFO:Medusa - Collective MF:[46] Solving for constraint matrices
INFO:Medusa - Collective MF:[47] Iteration
INFO:Medusa - Collec

X reconstruction -> X_prime: 
('x_id: ', 'X2')
('x_id: ', 'X3')
('x_id: ', 'X0')
('x_id: ', 'X1')
('x_id: ', 'X4')
('fname_out: ', './../../../sample_data_NCMF/ncmf_sim_data/dfmf/out/dn3/dict_out_dfmf.pkl')


In [6]:
print("done")

done
